<h1>pip</h1>

In [1]:
# Install necessary libraries
%pip install torch torchvision opencv-python matplotlib facenet-pytorch moviepy librosa transformers mediapipe

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0
  Rolling back uninstall of numpy
  Moving to c:\users\harshal shah\appdata\roaming\python\python312\scripts\f2py.exe
   from C:\Users\Harshal Shah\AppData\Local\Temp\pip-uninstall-mdl7bs3e\f2py.exe
  Moving to c:\users\harshal shah\appdata\roaming\python\python312\scripts\numpy-config.exe
   from C:\Users\Harshal Shah\AppData\Local\Temp\pip-uninstall-mdl7bs3e\numpy-config.exe
  Moving to c:\users\harshal shah\appdata\roaming\python\python312\site-packages\numpy-2.1.0-cp312-cp312-win_amd64.whl
   from C:\Users\Harshal Shah\AppData\Local\Temp\pip-uninstall-qw85czkp\numpy-2.1.0-cp312-cp312-win_amd64.whl
  Moving to c:\users\harshal shah\appdata\roaming\python\python312\site-packages\numpy-2.1.0.dist-info\
   fro

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\f2py.exe' -> 'c:\\Python312\\Scripts\\f2py.exe.deleteme'



<h1>Dependencies</h1>

In [2]:
import torch
import torchvision.models as models
import cv2
import numpy as np
from scipy.spatial.distance import cosine
from torchvision import transforms
import torchvision.transforms as transforms
from facenet_pytorch import MTCNN
from PIL import Image
import os
import librosa
from moviepy import VideoFileClip
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import mediapipe as mp
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

C:\Users\Harshal Shah\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


<h1>Face Distortion</h1>

<h4>Loading pre trained model</h4>

In [3]:
# Load MTCNN for face detection
mtcnn = MTCNN(keep_all=True, device=device)

# Load a pre-trained MobileNetV2 model for deepfake detection
mobilenet_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True).to(device)
mobilenet_model.eval()

# Modify the final layer for binary classification (Real/Fake)
num_ftrs = mobilenet_model.classifier[1].in_features
mobilenet_model.classifier[1] = torch.nn.Linear(num_ftrs, 2).to(device)

# Define preprocessing transformations for MobileNetV2
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the input size of MobileNetV2
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image
])

Using cache found in C:\Users\Harshal Shah/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\Harshal Shah\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Harshal Shah\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<h4>Detection</h4>

In [4]:
# Function to detect deepfakes in real-time
def detect_face_distortion(video_path, skip_frames=5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video at path {video_path}")
        return 0, 0

    frame_count = 0
    total_frames = 0
    distorted_faces = 0
    example_abnormal_frame = None  # To store one example of an abnormal frame

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames to reduce processing load
        frame_count += 1
        if frame_count % skip_frames != 0:
            continue

        # Increment total frame count
        total_frames += 1

        # Convert frame to RGB for MTCNN
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces using MTCNN
        boxes, _ = mtcnn.detect(rgb_frame)
        if boxes is None:
            continue

        # Process each detected face
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            face = rgb_frame[y1:y2, x1:x2]

            # Preprocess the face for MobileNetV2
            face_pil = Image.fromarray(face)
            input_tensor = transform(face_pil).unsqueeze(0).to(device)

            # Perform inference with MobileNetV2
            with torch.no_grad():
                output = mobilenet_model(input_tensor)
                _, predicted = torch.max(output, 1)
                prediction = "Real" if predicted.item() == 0 else "Fake"

            # If distortion (deepfake) is detected
            if prediction == "Fake":
                distorted_faces += 1

                # Save one example of an abnormal frame
                if example_abnormal_frame is None:
                    example_abnormal_frame = frame.copy()

                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, "Distorted Face", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    cap.release()

    # Display one example of an abnormal frame
    if example_abnormal_frame is not None:
        print("Displaying one example of an abnormal frame:")
        cv2.imshow("Example Abnormal Frame", example_abnormal_frame)
    else:
        print("No abnormal frames detected.")

    return total_frames, distorted_faces

<h4>Testing</h4>

In [5]:
# First, initialize MTCNN
mtcnn = MTCNN(keep_all=True, device=device)

# Then run your test
video_path = '000471.mp4'
skip_frames = 5  # Process every 5th frame to improve performance

total_frames, distorted_faces = detect_face_distortion(video_path, skip_frames)

# Print the results
print(f"Total Frames Processed: {total_frames}")
print(f"Total Distorted Faces Detected: {distorted_faces}")

No abnormal frames detected.
Total Frames Processed: 25
Total Distorted Faces Detected: 0


In [7]:
import json

# Define the path for the results.json file
results_file_path = 'results.json'

# Prepare the data to be saved
results_data = {
    'total_frames': total_frames,
    'distorted_faces': distorted_faces
}

try:
    # If the file exists and is not empty, read and update existing data
    if os.path.exists(results_file_path) and os.path.getsize(results_file_path) > 0:
        with open(results_file_path, 'r') as file:
            existing_data = json.load(file)
        # Update the existing data with new results
        existing_data.update(results_data)
        results_data = existing_data
    
    # Write the results to the file
    with open(results_file_path, 'w') as file:
        json.dump(results_data, file, indent=4)
    
    print(f"Results saved to {results_file_path}: {results_data}")

except Exception as e:
    print(f"Error handling results file: {str(e)}")

Results saved to results.json: {'total_frames': 25, 'distorted_faces': 0, 'total_frames_processed': 173, 'total_abnormal_frames_detected': 6, 'face_detection_rate': 97.47, 'cosine_similarity': -0.03, 'mismatch_score': 1.03, 'euclidean_distance': 1.43, 'analysis_result': 'High mismatch detected. Audio and visual content are inconsistent.'}
